# NHANES - Prevalence statistics

## Load libraries

In [2]:
library(dplyr)
library(ggplot2)
library(survey)
library(table1);

## Load data

In [3]:
nhanes <- read.csv("data/nhanes.csv")

## Set up survey design object


In [4]:
design <- svydesign(ids = ~masked.variance.psu, 
                           strata = ~masked.variance.stratum, 
                           weights = ~MEC10YR,
                           nest = TRUE,
                           data = nhanes)

In [5]:
# View the survey design object
print(design)

Stratified 1 - level Cluster Sampling design (with replacement)
With (153) clusters.
svydesign(ids = ~masked.variance.psu, strata = ~masked.variance.stratum, 
    weights = ~MEC10YR, nest = TRUE, data = nhanes)


## Calculate N & median values for TFR and FT by pregnancy_status

### TfR

#### N

In [ ]:
# N for ID (TfR >= 8.3 mg/l) by pregnancy status
n_id_tfr_by_pregnancy <- nhanes %>%
  group_by(pregnancy.status) %>%
  summarise(ID_tfr = sum(ID_tfr == 1))

# Print the results
print(n_id_tfr_by_pregnancy)

#### Medians

In [ ]:
# TfR medians by pregnancy status
medians <- svyby(~tfr, by = ~pregnancy.status, design, svyquantile, c(0.5), na.rm = TRUE)
print(medians)

In [ ]:
# Medians by depression status and pregnancy status
medians_by_pregnancy_and_depression <- svyby(~tfr, by = ~pregnancy.status + depression, 
                                             design, svyquantile, c(0.5), na.rm = TRUE)

print(medians_by_pregnancy_and_depression)

#### Prevalence

In [ ]:
# Calculate the prevalence of tfr >= 8.3 for each pregnancy status category
prevalence_by_pregnancy_status <- svyby(~ID_tfr, ~pregnancy.status, design, svymean, na.rm = TRUE)


In [ ]:
# Print the results
print(prevalence_by_pregnancy_status)

In [ ]:
prevalence_by_pregnancy_and_depression <- svyby(~ID_tfr, 
                                                ~pregnancy.status + depression, 
                                                design, svymean, na.rm = TRUE)


In [ ]:
# Print the results
print(prevalence_by_pregnancy_and_depression)

### Rao Scott Chi-Square test

#### ID TfR >= 8.3 Prevalence

In [ ]:
# Create a new design object for non-pregnant group (pregnancy.status == 0)
design_not_pregnant <- subset(design, pregnancy.status == 0)

In [ ]:
# Not pregnant group: Perform the Rao-Scott chi-squared test
result <- svychisq(~ID_tfr + depression, design = design_not_pregnant)

# Print the test result
print(result)

In [ ]:
# Create a new design object for pregnant group (pregnancy.status == 1)
design_pregnant <- subset(design, pregnancy.status == 1)

In [ ]:
# Pregnant group: Perform the Rao-Scott chi-squared test
result <- svychisq(~ID_tfr + depression, design = design_pregnant)

# Print the test result
print(result)

In [ ]:
# Create a new design object for postpartum group (pregnancy.status == 2)
design_postpartum <- subset(design, pregnancy.status == 2)

In [ ]:
# Postpartum group: Perform the Rao-Scott chi-squared test
result <- svychisq(~ID_tfr + depression, design = design_postpartum)

# Print the test result
print(result)

### Ferritin

In [6]:
# N for low ferritin by pregnancy status 
n_id_ferr_by_pregnancy <- nhanes %>%
  group_by(pregnancy.status) %>%
  summarise(ID_ferr = sum(ID_ferr == 1))

# Print the results
print(n_id_ferr_by_pregnancy)

# A tibble: 3 × 2
  pregnancy.status ID_ferr
             <dbl>   <int>
1                0     743
2                1     150
3                2      70


#### Medians

In [7]:
# ferritin medians by pregnancy status
medians_ferr <- svyby(~ferritin, by = ~pregnancy.status, design, svyquantile, c(0.5), na.rm = TRUE)
print(medians_ferr)

  pregnancy.status ferritin se.ferritin
0                0 46.61798   0.7322269
1                1 30.43630   1.7497517
2                2 37.43045   1.8812405


In [8]:
# Medians by depression status and pregnancy status
medians_by_pregnancy_and_depression_ferr <- svyby(~ferritin, by = ~pregnancy.status + depression, 
                                             design, svyquantile, c(0.5), na.rm = TRUE)

print(medians_by_pregnancy_and_depression_ferr)

    pregnancy.status depression ferritin se.ferritin
0.0                0          0 46.61798   0.6502434
1.0                1          0 30.43630   1.8282070
2.0                2          0 37.43045   2.0447228
0.1                0          1 49.21090   3.2154018
1.1                1          1 35.12984   8.1085087
2.1                2          1 31.51530   6.2605453


#### Prevalence

In [9]:
# Calculate the prevalence of ID ferr  for each pregnancy status category
prevalence_by_pregnancy_status_ferr <- svyby(~ID_ferr, ~pregnancy.status, 
                                             design, svymean, na.rm = TRUE)
print(prevalence_by_pregnancy_status_ferr)

  pregnancy.status   ID_ferr         se
0                0 0.1253447 0.00451530
1                1 0.2268897 0.02593807
2                2 0.1622995 0.02950532


In [10]:
prevalence_by_pregnancy_and_depression_ferr <- svyby(~ID_ferr, 
                                                ~pregnancy.status + depression, 
                                                design, svymean, na.rm = TRUE)

In [11]:
print(prevalence_by_pregnancy_and_depression_ferr)

    pregnancy.status depression   ID_ferr          se
0.0                0          0 0.1248613 0.004937327
1.0                1          0 0.2247528 0.026815083
2.0                2          0 0.1645061 0.030973769
0.1                0          1 0.1293349 0.014655558
1.1                1          1 0.2556869 0.075889190
2.1                2          1 0.1345304 0.062678824


In [12]:
# Create a new design object for non-pregnant group (pregnancy.status == 0)
design_not_pregnant <- subset(design, pregnancy.status == 0)

In [13]:
# Not Pregnant group: Perform the Rao-Scott chi-squared test
result <- svychisq(~ID_ferr + depression, design = design_not_pregnant)

# Print the test result
print(result)


	Pearson's X^2: Rao & Scott adjustment

data:  svychisq(~ID_ferr + depression, design = design_not_pregnant)
F = 0.080284, ndf = 1, ddf = 77, p-value = 0.7777



In [14]:
# Create a new design object for pregnant group (pregnancy.status == 1)
design_pregnant <- subset(design, pregnancy.status == 1)

In [15]:
# Pregnant group: Perform the Rao-Scott chi-squared test
result <- svychisq(~ID_ferr + depression, design = design_pregnant)

# Print the test result
print(result)


	Pearson's X^2: Rao & Scott adjustment

data:  svychisq(~ID_ferr + depression, design = design_pregnant)
F = 0.16677, ndf = 1, ddf = 62, p-value = 0.6844



In [16]:
# Create a new design object for postpartum group (pregnancy.status == 2)
design_postpartum <- subset(design, pregnancy.status == 2)

In [17]:
# Postpartum group: Perform the Rao-Scott chi-squared test
result <- svychisq(~ID_ferr + depression, design = design_postpartum)

# Print the test result
print(result)


	Pearson's X^2: Rao & Scott adjustment

data:  svychisq(~ID_ferr + depression, design = design_postpartum)
F = 0.17906, ndf = 1, ddf = 62, p-value = 0.6736

